# Generate Stimuli for Gaze Judgement Task

In [37]:
from matplotlib import pyplot as plt
from scipy import ndimage
import seaborn as sb
import pandas as pd
import imageio
import math
import numpy as np
import ffmpeg
import random
import os
import glob
import csv

# Note: gaze data collected at a rate of 250hz

# Grab batches of gaze data 

### get all free viewing data

In [38]:
def get_gaze(sub = 'pp151', this_list=1, this_version=1):

    path      = 'List'+str(this_list)+'/List'+str(this_list)+'_version'+str(this_version)+'/'
    p         = '../Free_viewing/'+path
    free_view = []

    for subject in [sub]:

        if subject!='.DS_Store' and subject!='.':
            subdir = p+subject+'/eye'

            for trial in os.listdir(subdir):
                dat_string = subdir+'/'+trial
                df         = pd.read_csv(dat_string, sep=" ", skiprows=None, skipinitialspace=True, skip_blank_lines=True, header=41)
                splitso    = df['Time\tType\tTrial\tL'].str.split('\t', expand=True)
                splitso    = splitso.rename(columns={0: "Time", 1: "Type", 2:"Trial", 3:"L POR X [px]",
                                                     4:"L POR Y [px]", 5:"R POR X [px]", 6:"R POR Y [px]",
                                                     7:"Timing", 8:"Pupil Confidence", 9:"L Plane", 10:"R Plane",
                                                     11:"L Event Info", 12:"R Event Info", 13:"Stimulus"})
                splitso = splitso[1:]
                splitso['L POR X [px]'] = splitso['L POR X [px]'].astype(float)
                splitso['L POR Y [px]'] = splitso['L POR Y [px]'].astype(float)
                splitso['subject']      = subject
                splitso['list']         = this_list
                splitso['version']      = this_version

                free_view.append(splitso)

        return(free_view)

In [39]:
free_view[0]['Stimulus'].unique()

NameError: name 'free_view' is not defined

# Get fixation chunk lists

In [40]:
def get_fixies(splitso):
    """
    Helper function to group fixation data from each trial into numbered chunks.
    Created by KZ.
    """
    events = list(splitso['L Event Info'])

    fixation_chunk = 1 
    fixie_list = []

    for idx,x in enumerate(events[:-1]):
        if x == 'Fixation':
            fixie_list.append(fixation_chunk)
            if events[idx+1] != 'Fixation':
                fixation_chunk += 1
    
    if events[-1] == 'Fixation':
        if events[-2] == 'Fixation':
            fixie_list.append(fixation_chunk)
        else:
            fixie_list.append(fixation_chunk+1)
                
    return fixie_list

In [41]:
free_fixies = [get_fixies(x) for x in free_view]

NameError: name 'free_view' is not defined

In [42]:
for x,y in zip(free_fixies,free_view):
    if len(x)!=y[y['L Event Info']=='Fixation'].shape[0]:
        print("Mismatch!")

NameError: name 'free_fixies' is not defined

### Get dictionaries with image and data tuples for visualizing

In [43]:
def make_the_dicts(splitso_list):

    dict_list = []
    
    xs = [x[x['L Event Info']=='Fixation']['L POR X [px]'] for x in splitso_list]
    ys = [x[x['L Event Info']=='Fixation']['L POR Y [px]'] for x in splitso_list]

    for idx,(x,y) in enumerate(zip(xs,ys)):
        tuple_list = [(np.round(a),np.round(b)) for a,b in zip(x,y)]
        the_dict = {'image':splitso_list[idx]['Stimulus'][1], 'fixations': tuple_list, 'subject':splitso_list[idx]['subject'].unique(),'list':splitso_list[idx]['list'].unique(), 'version':splitso_list[idx]['version'].unique()} 
        dict_list.append(the_dict)
        
    return dict_list

def make_full_dicts(splitso_list):

    dict_list = []
    
    xs = [x['L POR X [px]'] for x in splitso_list]
    ys = [x['L POR Y [px]'] for x in splitso_list]

    for idx,(x,y) in enumerate(zip(xs,ys)):
        tuple_list = [(np.round(a),np.round(b)) for a,b in zip(x,y)]
        the_dict = {'image':splitso_list[idx]['Stimulus'][1], 'gaze': tuple_list, 'subject':splitso_list[idx]['subject'].unique(),'list':splitso_list[idx]['list'].unique(), 'version':splitso_list[idx]['version'].unique()} 
        dict_list.append(the_dict)
        
    return dict_list

In [44]:
full_dicts = make_full_dicts(free_view)

NameError: name 'free_view' is not defined

In [45]:
full_dicts[0]

NameError: name 'full_dicts' is not defined

In [46]:
# free dicts is the data in dictionaries including fixations and saccades (all data)

### Check correct number of fixations for each 

# DIDEC Functions (from van Miltenberg et al., 2018)

In [23]:
# these functions are from : https://didec.uvt.nl/pages/download.html
# (see the second download link under "Images" section)
# any changed lines to the original functions have been commented "#KZ"

DISPLAY_SIZE = (1050, 1680)

def get_coords(entry):
    """
    Helper function to get coordinates from an entry.
    Own code.
    """
    x = round(float(entry['L POR X [px]']))
    y = round(float(entry['L POR Y [px]']))
    return x,y


def get_fixations(entries, remove_out_of_bounds=True):
    """
    Get the fixations from the BeGaze file, and return them in the PyGaze format.
    Own code.
    """
    coords = [get_coords(entry) for entry in entries
                                if entry['L Event Info'] == 'Fixation'
                                and entry['R Event Info'] == 'Fixation']
    if remove_out_of_bounds:
        height, width = DISPLAY_SIZE
        coords = [(x,y) for x,y in coords if x < width and y < height]
    return coords


def read_eye_data(filepath, remove_out_of_bounds=True):
    """
    Read the BeGaze data.
    Own code.
    """
    with open(filepath) as f:
        # There's a lot of lines that are just unnecessary..
        for i in range(41):
            _ = next(f) # Skip line
        reader = csv.DictReader(f, delimiter='\t')
        entries = list(reader)
        # The first entry is not a real entry but a message about the stimulus.
        message = entries[0]
        image = entries[0]['L POR X [px]'].split()[-1]
        del entries[0]
        return dict(image=image, fixations=get_fixations(entries, remove_out_of_bounds))


def buildFixMap(fixations, doBlur=True, sigma=19, display=(1050, 1680)):
    """
    Function to build a fixation map.
    Based on salicon.py
    """
    sal_map = np.zeros(display)
    for x,y in fixations:
        #if y<=1050.00 and x<=1680.00:
        sal_map[int(y)][int(x)]=1
            # KZ #sal_map[y][x] = 1 
    if doBlur:
        sal_map = ndimage.filters.gaussian_filter(sal_map, sigma)
        sal_map -= np.min(sal_map)
        sal_map /= np.max(sal_map)
    return sal_map


def clean_heatmap(heatmap):
    """
    Remove numbers below the mean to make the visualization clearer.
    Own code.
    """
    lowbound = np.mean(heatmap[heatmap>0])
    heatmap[heatmap<lowbound] = np.NaN
    return heatmap


def CC_score(gtsAnn, resAnn):
    """
    Computer CC score. A simple implementation
    From the SALICON codebase.
    
    :param gtsAnn : ground-truth fixation map
    :param resAnn : predicted saliency map
    :return score: int : score
    """
    fixationMap = gtsAnn - np.mean(gtsAnn)
    if np.max(fixationMap) > 0:
        fixationMap = fixationMap / np.std(fixationMap)
    salMap = resAnn - np.mean(resAnn)
    if np.max(salMap) > 0:
        salMap = salMap / np.std(salMap)
    
    return np.corrcoef(salMap.reshape(-1), fixationMap.reshape(-1))[0][1]


# KZ added map_only option #def plot_heatmap(data, filename='test.pdf', alpha=0.7, cmap='jet', clean=True):
def plot_heatmap(data, filename='test.pdf', alpha=0.7, cmap='jet', clean=True, map_only=False, other=None):
    """
    Plot heatmap.
    Mostly modified from PyGaze.
    """
    # Load data
    image      = data['image']
    image_path = '/Users/kirstenziman/Downloads/images_with_borders/Images_resized_greyborders/List 1/'+data['image'] # IMAGE_PATHS[image]
    #KZ #image_data = ndimage.imread(image_path)
    image_data = imageio.imread(image_path)
    heatmap    = buildFixMap(data['fixations'])
    # Remove haze.
    if clean:
        heatmap = clean_heatmap(heatmap)
    
    # Matplotlib.
    # Borrows heavily from from Edwin Dalmaijer's `gazeplotter.py` script, from the PyGaze codebase.
    dpi = 100
    display_size = heatmap.shape
    figsize = (display_size[1]/dpi, display_size[0]/dpi)
    fig = plt.figure(figsize=figsize, dpi=dpi, frameon=False)
    ax = plt.Axes(fig, [0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    # KZ #ax.imshow(image_data)
    
    #KZ added ax.imshow(image_data) inside if statement for "map_only" option
    if map_only == False and other == None:
        ax.imshow(image_data)
    
    elif other != None:
        image_data= imageio.imread(other)
        ax.imshow(image_data)
        print(other)
        
    ax.imshow(heatmap, cmap=cmap, alpha=alpha)
    #KZ #fig.savefig(filename)

### Make movie stimuli

Running this cell will make VERIDICAL, SCRAMBLED, MISMATCHED, and ISOLATED attention stimuli.

It will take a very long time to run to completion.

In [24]:
def movie_maker(fixie_lists, dicts, stim_type='veridical', map_only=False):
    '''
    inputs: lists of fixation chunks
            dictionaries of gaze dat
            
            stim_type : 'veridical','scrambled'
            map_only  : boolean
            
    output: mp4 video of the attention video stimuli
    '''
    
    for i,(fixie_list,the_dict) in enumerate(zip(fixie_lists,dicts)):

        lengths = []
        movie_frames = []
        total_counter = 0
        
        k = list(set(fixie_list))
        print(k)
        print(len(k))
        
        if stim_type=='scrambled':
            new_list  = [k[0]]
            remainder = k[1:]
            np.random.shuffle(remainder)
            for z in remainder:
                new_list.append(z)
            print(new_list)
            print(len(new_list))
            k = new_list
            
        # for each individual fixation chunk number
        for x in k:

            # length counter - count number of timepoints in this chunk
            length_counter = 0

            # empty list to collect tuples
            tuple_list = []

            # collect the total counter for this list only
            total_list = []

            # for each item in fixie chunk list AND each tuple
            for a,b in zip(fixie_list, the_dict['fixations']):

                # if item from fixie chunk list is the number from first for statement:
                if a == x:
                    
#                     DISPLAY_SIZE = (1050, 1680)
#                     if b[0]<=1050.00 and b[1]<=1680.00:

                    # add tuple to current tuple list
                    tuple_list.append(b)

                    # add one to the length counter
                    length_counter += 1

                    # add one to total counter
                    total_counter += 1

                    total_list.append(total_counter)

            lengths.append(length_counter)

            # pass tuple list into plotting function
            this_dict = {'image':the_dict['image'], 'fixations':tuple_list}
            path      = '/Users/kirstenziman/Downloads/images_with_borders/Images_resized_greyborders/List 1/'
            im_path   = path + this_dict['image']

            plot_heatmap(this_dict, filename=im_path, alpha=.6, cmap="Greys_r", clean=False, map_only=map_only, other=None)

            for p,q in zip(range(0,length_counter),total_list):
                if q<10:
                    addin = '_00'
                elif q>=10 and q<100:
                    addin = '_0'
                else:
                    addin = '_'

                plt.savefig(this_dict['image']+addin+str(q)+'.jpg')

        # determine frame rate
        num_images = 0
        for img in os.listdir('.'):
            if img[-3:]=='jpg':
                num_images+=1
        
        # total frames / total seconds  --> frames per second
        frame_rate = math.floor(num_images/3)
        
        if map_only == False:
            ending = the_dict['subject'][0]+'_L1V1_freeview_'+stim_type+'.mp4'
        else:
            ending = the_dict['subject'][0]+'_L1V1_freeview_'+stim_type+'_mapOnly.mp4'
        
        (
        ffmpeg
        .input('*.jpg', pattern_type='glob', framerate=frame_rate)
        .output(str(i)+'_'+the_dict['image']+ending)
        .run()
        )

        for file_name in os.listdir('.'):
            if file_name.endswith('.jpg'):
                os.remove(file_name)

In [25]:
def movie_maker_2(fixie_lists, dicts, stim_type='mismatch', map_only=False, other=None):
    '''
    inputs: lists of fixation chunks
            dictionaries of gaze dat
            
            stim_type : 'mismatch'
            map_only  : boolean
            
    output: mp4 video of the attention video stimuli
    '''
    
    for i,(fixie_list,the_dict) in enumerate(zip(fixie_lists,dicts)):

        lengths = []
        movie_frames  = []
        total_counter = 0
        
        k = list(set(fixie_list))
        print(k)
        print(len(k))
        
        if stim_type=='scrambled':
            new_list  = [k[0]]
            remainder = k[1:]
            np.random.shuffle(remainder)
            for z in remainder:
                new_list.append(z)
            print(new_list)
            print(len(new_list))
            k = new_list
            
        # for each individual fixation chunk number
        for x in k:

            # length counter - count number of timepoints in this chunk
            length_counter = 0

            # empty list to collect tuples
            tuple_list = []

            # collect the total counter for this list only
            total_list = []

            # for each item in fixie chunk list AND each tuple
            for a,b in zip(fixie_list, the_dict['fixations']):

                # if item from fixie chunk list is the number from first for statement:
                if a == x:
                    
#                     DISPLAY_SIZE = (1050, 1680)
#                     if b[0]<=1050.00 and b[1]<=1680.00:

                    # add tuple to current tuple list
                    tuple_list.append(b)

                    # add one to the length counter
                    length_counter += 1

                    # add one to total counter
                    total_counter += 1

                    total_list.append(total_counter)

            lengths.append(length_counter)

            # pass tuple list into plotting function
            this_dict = {'image':the_dict['image'], 'fixations':tuple_list}
            path      = '/Users/kirstenziman/Downloads/images_with_borders/Images_resized_greyborders/List 1/'
            
            im_path = path+other
            
#             if this_dict['image'] == '1592568.bmp':
#                 other = '150273.bmp'
#                 im_path = path + other #this_dict['image']
                
#             elif this_dict['image'] == '150273.bmp':
#                 other = '1592568.bmp'
#                 im_path = path + other

            plot_heatmap(this_dict, filename=im_path, alpha=.6, cmap="Greys_r", clean=False, map_only=map_only, other=im_path)

            for p,q in zip(range(0,length_counter),total_list):
                if q<10:
                    addin = '_00'
                elif q>=10 and q<100:
                    addin = '_0'
                else:
                    addin = '_'

                plt.savefig(this_dict['image']+addin+str(q)+'.jpg')

        # determine frame rate
        num_images = 0
        for img in os.listdir('.'):
            if img[-3:]=='jpg':
                num_images+=1
        
        # total frames / total seconds  --> frames per second
        frame_rate = math.floor(num_images/3)
        
        if map_only == False:
            ending = the_dict['subject'][0]+'_L1V1_freeview_'+stim_type+'.mp4'
        else:
            ending = the_dict['subject'][0]+'_L1V1_freeview_'+stim_type+'_mapOnly.mp4'
        
        (
        ffmpeg
        .input('*.jpg', pattern_type='glob', framerate=frame_rate)
        .output(str(i)+'_'+the_dict['image']+'_'+other+ending)
        .run()
        )

        for file_name in os.listdir('.'):
            if file_name.endswith('.jpg'):
                os.remove(file_name)

In [26]:
def movie_maker_3(dicts, stim_type='full', map_only=False, other=None):
    '''
    inputs: lists of fixation chunks
            dictionaries of gaze dat
            
            stim_type : 'mismatch'
            map_only  : boolean
            
    output: mp4 video of the attention video stimuli
    '''
    
    for the_dict in dicts:
        
        total_list = len(the_dict['gaze'])

        length_counter = 0
        # DISPLAY_SIZE = (1050, 1680)
            
        # for each individual fixation chunk number
        for x in the_dict['gaze']:
            
            
            centroid  = x
            
            fixations = [x]
            
            for a in range(1,10):
                fixations.append((x[0]+a, x[1]))
                fixations.append((x[0], x[1]+a))

            # pass tuple list into plotting function
            this_dict = {'image':the_dict['image'], 'fixations':fixations}
            path      = '/Users/kirstenziman/Downloads/images_with_borders/Images_resized_greyborders/List 1/'
            
            im_path = path+this_dict['image']
            

            plot_heatmap(this_dict, filename=im_path, alpha=.6, cmap="Greys_r", clean=False, map_only=map_only, other=None)

            #for p,q in zip(range(0,length_counter),total_list):
            if length_counter<10:
                addin = '_00'
            elif length_counter>=10 and length_counter<100:
                addin = '_0'
            else:
                addin = '_'

            plt.savefig(this_dict['image']+addin+str(length_counter)+'.jpg')
                
            length_counter+=1

        # determine frame rate
        num_images = 0
        for img in os.listdir('.'):
            if img[-3:]=='jpg':
                num_images+=1
        
        # total frames / total seconds  --> frames per second
        frame_rate = math.floor(num_images/3)
        
        if map_only == False:
            ending = the_dict['subject'][0]+'_L1V1_freeview_'+stim_type+'_FULL.mp4'
        else:
            ending = the_dict['subject'][0]+'_L1V1_freeview_'+stim_type+'_FULL_mapOnly.mp4'
        
        (
        ffmpeg
        .input('*.jpg', pattern_type='glob', framerate=frame_rate)
        .output(str(i)+'_'+the_dict['image']+'_'+other+ending)
        .run()
        )

        for file_name in os.listdir('.'):
            if file_name.endswith('.jpg'):
                os.remove(file_name)

In [27]:
#ffmpeg -i thisImage_FULL.mp4 -vf reverse reversedFULL.mp4

In [28]:
#         (
#         ffmpeg
#         .input('*.jpg', pattern_type='glob', framerate=250)
#         .output(str(0)+'_'+'thisImage'+'_'+'FULL'+'.mp4')
#         .run()
#         )

#         for file_name in os.listdir('.'):
#             if file_name.endswith('.jpg'):
#                 os.remove(file_name)

### Make the movies!

In [29]:
movie_maker_3(full_dicts[2:4], stim_type='full', map_only=False, other=None)

NameError: name 'full_dicts' is not defined

In [30]:
### REMEMBER TO CHANGE BACK COUNTER IN MOVIE MAKING FUNCTION BEFORE RE-RUNNING !!!

# movie_maker(free_fixies[2:4], free_dicts[0:2], stim_type='veridical')
# movie_maker(free_fixies[2:4], free_dicts[0:2], stim_type='scrambled')
# movie_maker(free_fixies[0:2], free_dicts[0:2], stim_type='veridical',map_only=True)
# movie_maker(free_fixies[0:2], free_dicts[0:2], stim_type='scrambled',map_only=True)

# for x,y in zip([0,1],['1592286.bmp','150273.bmp']):

#     movie_maker_2(free_fixies[x:x+1], free_dicts[x:x+1], stim_type='mismatch',map_only=False, other=y)